In [1]:
!pip install selenium

In [3]:
!pip install beautifulsoup4

In [4]:
## importing libraries
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [6]:
## Linkedin ID and PASSWORD
email = "write your email here"
password = "write your password here"

In [7]:
## Open browser
## You first need to download chrome driver
driver_path = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
## Maximizing browser window to avoid hidden elements
driver.set_window_size(1024, 600)
driver.maximize_window()

## Open linkedin website
driver.get('https://www.linkedin.com/login')
## waiting load
time.sleep(2)

## Search for login and password inputs, send credentions 
driver.find_element_by_id('username').send_keys(email)
driver.find_element_by_id('password').send_keys(password)
driver.find_element_by_id('password').send_keys(Keys.RETURN)

## Opening jobs webpage
## You can change the link by copying the link to the jobs you want to scrape after typing the title and location
driver.get(f"https://www.linkedin.com/jobs/search/?geoId=91000000&keywords=junior%20data%20scientist&location=Union%20europ%C3%A9enne")
## waiting load
time.sleep(2)

<ipython-input-7-b276dae96912>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)
<ipython-input-7-b276dae96912>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('username').send_keys(email)
<ipython-input-7-b276dae96912>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('password').send_keys(password)
<ipython-input-7-b276dae96912>:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('password').send_keys(Keys.RETURN)


In [8]:
## Create a list where the descriptions will be stored
desc_list = []

## each page show us some jobs, sometimes show 25, others 13 or 21 ¯\_(ツ)_/¯
## with this knowledge I created a loop that will check how many jobs the page is listing
## Then 

## linkedin show us 40 jobs pages, then the line below will repeat 40 times

for i in range(1,41):
    ## click button to change the job list
    driver.find_element_by_xpath(f'//button[@aria-label="Page {i}"]').click()
    ## each page show us some jobs, sometimes show 25, others 13 or 21 ¯\_(ツ)_/¯
    jobs_lists = driver.find_element_by_class_name('jobs-search-results__list') #here we create a list with jobs
    jobs = jobs_lists.find_elements_by_class_name('jobs-search-results__list-item')#here we select each job to count
    ## waiting load
    time.sleep(1) 
    ## the loop below is for the algorithm to click exactly on the number of jobs that is showing in list
    ## in order to avoid errors that will stop the automation
    for job in range (1, len(jobs)+1):
        ## job click
        driver.find_element_by_xpath(f'//div[@aria-label="search results"]/div/ul/li[{job}]/div/div[1]/div[1]/div[2]/div[1]/a').click()
        ## waiting load 
        time.sleep(2)
        ## select size and type of the company
        size_type = driver.find_element_by_xpath(f'//div[@class="mt5 mb2"]/ul/li[2]/span')
        ## select job description
        job_desc = driver.find_element_by_id('job-details')
        ## get text
        soup1 = BeautifulSoup(size_type.get_attribute('outerHTML'), 'html.parser')
        soup2 = BeautifulSoup(job_desc.get_attribute('outerHTML'), 'html.parser')
        ## add text to list
        desc_list.append([soup1.text, soup2.text])

<ipython-input-8-5835f3e62e70>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(f'//button[@aria-label="Page {i}"]').click()
<ipython-input-8-5835f3e62e70>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  jobs_lists = driver.find_element_by_class_name('jobs-search-results__list') #here we create a list with jobs
C:\Users\user\anaconda\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
<ipython-input-8-5835f3e62e70>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(f'//div[@aria-label="search results"]/div/ul/li[{job}]/div/div[1]/div[1]/div[2]/div[1]/a').click()
<ipython-in

In [9]:
## Count the number of jobs scraped
len(desc_list)

459

In [13]:
import pandas as pd
## Create dictionnary
data = {'job_description': [x[1] for x in disc_list],
        'size_type': [x[0] for x in disc_list]}
 
## Create DataFrame
df = pd.DataFrame(data)

In [14]:
## Read the dataset
df.head()

,job_description,size_type
0,\n\n\n \n We are proud to belong ...,\n201-500 employees · Computer Games\n
1,\n\n\n \nJob Description\n\nJob Description\n ...,"\n5,001-10,000 employees · Motor Vehicle Manuf..."
2,\n\n\n \n This role is accountabl...,"\n10,001+ employees · Telecommunications\n"
3,\n\n\n \n Everything happens some...,\n51-200 employees · Internet Publishing\n
4,\n\n\n \nJob Responsibilities\n Process data a...,"\n10,001+ employees · Semiconductor Manufactur..."


In [ ]:
## Split size and type and transform them to 2 separated columns
df = pd.concat([df, df['size_type'].str.split(pat = '·', expand = True)], axis=1)
## Rename the new columns
df.rename(columns = {0:'size', 1:'activity'}, inplace = True)
## Drop old column
df.drop(['size_type'], axis=1, inplace=True)
## Drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
## Save csv
df.to_csv('jobs.csv')